### Classification based on extracted features

In [2]:
import tensorflow as tf
tf.test.is_gpu_available()

True

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

#from keras.applications.vgg16 import VGG16 as BaseModel
#from keras.applications.vgg16 import preprocess_input

from tensorflow.keras.applications.xception import Xception as BaseModel
from tensorflow.keras.applications.xception import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import numpy as np

In [4]:
model = BaseModel(weights='imagenet', include_top=True)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [6]:
features_model = Model(model.input, model.layers[-2].output)

In [7]:
features_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [8]:
test_dir = 'data/training_folder/test'
val_dir = 'data/training_folder/val'
train_dir = 'data/training_folder/train'
image_size = tuple(model.input.shape[1:3].as_list()) ###(224,224)

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) 
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) 
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) 

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=image_size,
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=image_size,
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=image_size,
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

test_filenames = test_generator.filenames
val_filenames = val_generator.filenames
train_filenames = train_generator.filenames

test_nb_samples = len(test_filenames)
val_nb_samples = len(val_filenames)
train_nb_samples = len(train_filenames)

predict_test = features_model.predict_generator(test_generator,steps=test_nb_samples)
predict_val = features_model.predict_generator(val_generator,steps=val_nb_samples)
predict_train = features_model.predict_generator(train_generator,steps=train_nb_samples)

Found 340 images belonging to 17 classes.
Found 340 images belonging to 17 classes.
Found 680 images belonging to 17 classes.


In [10]:
predict_train.shape

(680, 2048)

Now we'll use classical ML algorithm on extracted features

Let us use only the first two classes

In [11]:
features_test = predict_test[test_generator.classes <=1]
features_train = predict_val[val_generator.classes <=1]

y_test = test_generator.classes[test_generator.classes <=1]
y_train = val_generator.classes[val_generator.classes <=1]

In [12]:
from sklearn.linear_model import LogisticRegression

In [13]:
clf = LogisticRegression()

In [14]:
clf.fit(features_train, y_train)

LogisticRegression()

In [15]:
pred = clf.predict(features_test)

In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(pred, y_test)

0.95

For multiple classes

In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [18]:
n_neighbors = 20

In [19]:
#clf = KNeighborsClassifier(n_neighbors=n_neighbors)
#clf = RandomForestClassifier(random_state=42)
clf = MLPClassifier()

In [20]:
features_test = predict_test
features_train = predict_train

y_test = test_generator.classes
y_train = train_generator.classes

In [21]:
clf.fit(features_train, y_train)

MLPClassifier()

In [22]:
pred = clf.predict(features_test)

In [23]:
accuracy_score(pred, y_test)

0.9205882352941176